In [1]:
import pandas as pd
import os
import numpy as np
from IPython.display import display


In [2]:
# Raw files
school_data = pd.read_csv(R'Resources\schools_complete.csv')
student_data = pd.read_csv(R'Resources\students_complete.csv')

# Checking for missing data.
print(school_data.isnull().sum())
print(student_data.isnull().sum())

School ID      0
school_name    0
type           0
size           0
budget         0
dtype: int64
Student ID       0
student_name     0
gender           0
grade            0
school_name      0
reading_score    0
math_score       0
dtype: int64


### Cleanup

In [3]:
# Student names has prefixes and suffixes so I need to remove those.
student_names = student_data['student_name'].tolist()

students_to_fix = []
for name in student_names:
    if len(name.split()) >= 3:
        students_to_fix.append(name)

prefixes = []
for name in students_to_fix:
    if len(name.split()[0]) <= 4:
        if name.split()[0] not in prefixes:
            prefixes.append(name.split()[0])
        else:
            pass
print(prefixes)

suffixes = []
for name in students_to_fix:
    if len(name.split()[-1]) <= 3:
        if name.split()[-1] not in suffixes:
            suffixes.append(name.split()[-1])
        else:
            pass
print(suffixes)

prefixes_suffixes = ['Dr. ', 'Mr. ','Ms. ', 'Mrs. ', 'Miss ', ' MD', ' DDS', ' DVM', ' PhD']

['Dr.', 'Mr.', 'Miss', 'Luke', 'Mrs.', 'Mary', 'Tara', 'John', 'Dale', 'Ms.', 'Amy', 'Lisa', 'Mark', 'Cody', 'Sara', 'Ryan', 'Tony', 'Jodi', 'Ruth', 'Adam', 'Eric', 'Kyle', 'Lynn', 'Jon', 'Judy', 'Tina', 'Paul', 'Marc', 'Jill', 'Carl', 'Jose', 'Emma', 'Gary', 'Lori', 'Noah', 'Dawn', 'Chad', 'Anne', 'Troy', 'Anna', 'Mike', 'Todd', 'Leah', 'Ian', 'Kari', 'Kara', 'Cory', 'Erin', 'Greg', 'Joe', 'Dana', 'Erik', 'Gail', 'Omar', 'Seth', 'Gina', 'Sean', 'Toni', 'Kim']
['MD', 'III', 'DVM', 'DDS', 'II', 'PhD', 'Lee', 'Jr.', 'IV', 'Cox', 'Roy', 'Day', 'V', 'Kim', 'Li']


In [4]:
# Cleaning the names
for word in prefixes_suffixes:
    student_data['student_name'] = student_data['student_name'].str.replace(word,'')

# Done cleaning
cleaned_file_path = R'Resources\cleaned_students.csv'
if os.path.exists(cleaned_file_path):
    os.remove(cleaned_file_path)
    student_data.to_csv(cleaned_file_path)
else:
    student_data.to_csv(cleaned_file_path)

# Create one dataframe
cleaned_student_data = pd.read_csv(R'Resources\students_complete.csv')
school_student_df = pd.merge(cleaned_student_data, school_data, on='school_name')

<ipython-input-4-73dffcfbe57f>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  student_data['student_name'] = student_data['student_name'].str.replace(word,'')


### General analysis

In [5]:
# Number of students
student_count = school_student_df['Student ID'].count()

# Number of schools
school_count = len(school_student_df['school_name'].unique())

# Budget
total_budget = school_data['budget'].sum()

# Averages
average_reading_score = school_student_df['reading_score'].mean()
average_math_score = school_student_df['math_score'].mean()

# Percentages
passing_math_count = school_student_df[school_student_df['math_score'] >= 70].count()[0]
passing_reading_count = school_student_df[school_student_df['reading_score'] >= 70].count()[0]
passing_math_reading_count = school_student_df[(school_student_df['math_score'] >= 70) & (school_student_df['reading_score'] >= 70)].count()[0]

passing_math_percentage = passing_math_count / student_count * 100
passing_reading_percentage = passing_reading_count / student_count * 100
overall_passing_percentage = passing_math_reading_count / student_count * 100



In [6]:
# District statistical analysis
district_summary_df = pd.DataFrame(
          [{'Total Schools': school_count,
          'Total Students': student_count,
          'Total Budget': total_budget,
          'Average Math Score': average_math_score,
          'Average Reading Score': average_reading_score,
          '% Passing Math': passing_math_percentage,
         '% Passing Reading': passing_reading_percentage,
        '% Overall Passing': overall_passing_percentage}])
district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,65.172326


In [7]:
# Some formatting
district_summary_df['Total Students'] = district_summary_df['Total Students'].map('{:,}'.format)
district_summary_df['Total Budget'] = district_summary_df['Total Budget'].map('${:,.2f}'.format)
district_summary_df[['Average Math Score', 'Average Reading Score']] = district_summary_df[['Average Math Score', 'Average Reading Score']].applymap('{:.1f}'.format)
district_summary_df[['% Passing Math', '% Passing Reading', '% Overall Passing']] = district_summary_df[['% Passing Math', '% Passing Reading', '% Overall Passing']].applymap('{:.0f}'.format)
district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",79.0,81.9,75,86,65


In [8]:
# Per school statistical analysis

# Types of schools
per_school_types = school_data.set_index(['school_name'])['type']

# Capita
per_school_counts = school_student_df['school_name'].value_counts()
per_school_budget = school_data.set_index(['school_name'])['budget']
per_school_capita = per_school_budget / per_school_counts

# Per school aAverages
per_school_math = school_student_df.groupby(['school_name']).mean()['math_score']
per_school_reading = school_student_df.groupby(['school_name']).mean()['reading_score']

# Per school passing
per_school_passing_math = school_student_df[(school_student_df['math_score'] >= 70)]
per_school_passing_math = per_school_passing_math.groupby(['school_name']).count()['student_name']
per_school_passing_math  = per_school_passing_math / per_school_counts * 100

per_school_passing_reading = school_student_df[(school_student_df['reading_score'] >= 70)]
per_school_passing_reading = per_school_passing_reading.groupby(['school_name']).count()['student_name']
per_school_passing_reading = per_school_passing_reading / per_school_counts * 100

per_passing_math_reading = school_student_df[(school_student_df['math_score'] >= 70) & (school_student_df['reading_score'] >= 70)]
per_passing_math_reading = per_passing_math_reading.groupby(['school_name']).count()['student_name']

per_overall_passing_percentage = per_passing_math_reading / per_school_counts * 100

In [9]:
per_school_counts

Bailey High School       4976
Johnson High School      4761
Hernandez High School    4635
Rodriguez High School    3999
Figueroa High School     2949
Huang High School        2917
Ford High School         2739
Wilson High School       2283
Cabrera High School      1858
Wright High School       1800
Shelton High School      1761
Thomas High School       1635
Griffin High School      1468
Pena High School          962
Holden High School        427
Name: school_name, dtype: int64

In [10]:
# Per school statistical analysis
per_school_summary_df = pd.DataFrame({
              'School Type': per_school_types,
              'Total Students': per_school_counts,
              'Total School Budget': per_school_budget,
              'Per Student Budget': per_school_capita,
              'Average Math Score': per_school_math,
              'Average Reading Score': per_school_reading,
              '% Passing Math': per_school_passing_math,
              '% Passing Reading': per_school_passing_reading,
              '% Overall Passing': per_overall_passing_percentage})
per_school_summary_df.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,90.599455


In [11]:
# Some formatting
per_school_summary_grades = per_school_summary_df.copy()
per_school_summary_grades[['Total School Budget', 'Per Student Budget']] = per_school_summary_grades[['Total School Budget', 'Per Student Budget']].applymap('${:,.2f}'.format)
per_school_summary_grades['Total Students'] = per_school_summary_grades['Total Students'].map('{:,}'.format)
per_school_summary_grades[['Average Math Score', 'Average Reading Score']] = per_school_summary_grades[['Average Math Score', 'Average Reading Score']].applymap('{:.1f}'.format)
per_school_summary_grades[['% Passing Math', '% Passing Reading', '% Overall Passing']] = per_school_summary_grades[['% Passing Math', '% Passing Reading', '% Overall Passing']].applymap('{:.0f}'.format)
per_school_summary_grades

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.0,81.0,67,82,55
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.1,84.0,94,97,91
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.7,81.2,66,81,53
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.1,80.7,68,79,54
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.4,83.8,93,97,91
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.3,80.9,67,81,54
Holden High School,Charter,427,"$248,087.00",$581.00,83.8,83.8,93,96,89
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.6,81.2,66,81,54
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.1,81.0,66,81,54
Pena High School,Charter,962,"$585,858.00",$609.00,83.8,84.0,95,96,91


In [12]:
top_schools = per_school_summary_grades.sort_values(['% Overall Passing'], ascending=False).head(5)
bottom_schools = per_school_summary_grades.sort_values(['% Overall Passing'], ascending=False).tail(5)

display(top_schools)
bottom_schools

NameError: name 'disptop_schools' is not defined

In [ ]:
# Per grade level analysis
ninth_graders = school_student_df[(school_student_df['grade'] == '9th')]
tenth_graders = school_student_df[(school_student_df['grade'] == '10th')]
eleventh_graders = school_student_df[(school_student_df['grade'] == '11th')]
twelfth_graders = school_student_df[(school_student_df['grade'] == '12th')]

# Math scores by grade
ninth_grade_math_scores = ninth_graders.groupby(['school_name']).mean()['math_score']
tenth_grade_math_scores = tenth_graders.groupby(['school_name']).mean()['math_score']
eleventh_grade_math_scores = eleventh_graders.groupby(['school_name']).mean()['math_score']
twelfth_grade_math_scores = twelfth_graders.groupby(['school_name']).mean()['math_score']

# Reading scores by grade
ninth_grade_reading_scores = ninth_graders.groupby(['school_name']).mean()['reading_score']
tenth_grade_reading_scores = tenth_graders.groupby(['school_name']).mean()['reading_score']
eleventh_grade_reading_scores = eleventh_graders.groupby(['school_name']).mean()['reading_score']
twelfth_grade_reading_scores = twelfth_graders.groupby(['school_name']).mean()['reading_score']

In [ ]:
scores_by_grade = pd.DataFrame({
                '9th Grade Math': ninth_grade_math_scores,
                '9th Grade Reading': ninth_grade_reading_scores, 
                '10th Grade Math': tenth_grade_math_scores,
                '10th Grade Reading': tenth_grade_reading_scores,
                '11th Grade Math': eleventh_grade_math_scores,
                '11th Grade Reading': eleventh_grade_reading_scores, 
                '12th Grade Math': twelfth_grade_math_scores,
                '12th Grade Reading': twelfth_grade_reading_scores})
                
scores_by_grade = scores_by_grade.applymap('{:.1f}'.format)

scores_by_grade.head()

,9th Grade Math,9th Grade Reading,10th Grade Math,10th Grade Reading,11th Grade Math,11th Grade Reading,12th Grade Math,12th Grade Reading
school_name,,,,,,,,
Bailey High School,77.1,81.3,77.0,80.9,77.5,80.9,76.5,80.9
Cabrera High School,83.1,83.7,83.2,84.3,82.8,83.8,83.3,84.3
Figueroa High School,76.4,81.2,76.5,81.4,76.9,80.6,77.2,81.4
Ford High School,77.4,80.6,77.7,81.3,76.9,80.4,76.2,80.7
Griffin High School,82.0,83.4,84.2,83.7,83.8,84.3,83.4,84.0


In [ ]:
def formatting_columns(df):
    df[['Average Math Score', 'Average Reading Score']] = df[['Average Math Score', 'Average Reading Score']].applymap('{:.1f}'.format)
    df[['% Passing Math', '% Passing Reading', '% Overall Passing']] = df[['% Passing Math', '% Passing Reading', '% Overall Passing']].applymap('{:.0f}'.format)
    return df

def create_summary_dataframe(input_df, col):
    avg_math = input_df.groupby([col]).mean()['Average Math Score']
    avg_reading = input_df.groupby([col]).mean()['Average Reading Score']
    perc_avg_math = input_df.groupby([col]).mean()['% Passing Math']
    perc_avg_reading = input_df.groupby([col]).mean()['% Passing Reading']
    perc_overall = input_df.groupby([col]).mean()['% Overall Passing']

    output_df = pd.DataFrame({
          'Average Math Score' : avg_math,
          'Average Reading Score': avg_reading,
          '% Passing Math': perc_avg_math,
          '% Passing Reading': perc_avg_reading,
          '% Overall Passing': perc_overall})

    output_df[['Average Math Score', 'Average Reading Score']] = output_df[['Average Math Score', 'Average Reading Score']].applymap('{:.1f}'.format)
    output_df[['% Passing Math', '% Passing Reading', '% Overall Passing']] = output_df[['% Passing Math', '% Passing Reading', '% Overall Passing']].applymap('{:.0f}'.format)

    return output_df

In [ ]:
# Spending ranges per Student
per_school_capita.describe()
spending_bins = [0, 585, 630, 645, 675]
group_names = ['<$586', '$586-630', '$631-645', '$646-675']

# per_school_capita.groupby(pd.cut(per_school_capita, spending_bins)).count()
per_school_summary_df['Spending Ranges (Per Student)'] = pd.cut(per_school_capita, spending_bins, labels=group_names)

# Create spending summary
spending_summary_df = create_summary_dataframe(per_school_summary_df, 'Spending Ranges (Per Student)')
spending_summary_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$586,83.5,83.9,93,97,90
$586-630,81.9,83.2,87,93,81
$631-645,78.5,81.6,73,84,63
$646-675,77.0,81.0,66,81,54


In [ ]:
# School size analysis
size_bins = [0, 999, 1999, 5000]
group_names = ['Small (<1000)', 'Medium (1000-1999)', 'Large (2000-5000)']
per_school_summary_df['School Size'] = pd.cut(per_school_summary_df['Total Students'], size_bins, labels=group_names)

# Create size summary
size_summary_df = create_summary_dataframe(per_school_summary_df, 'School Size')
size_summary_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.8,83.9,94,96,90
Medium (1000-1999),83.4,83.9,94,97,91
Large (2000-5000),77.7,81.3,70,83,58


In [ ]:
# School type summary
type_summary_df = create_summary_dataframe(per_school_summary_df, 'School Type')
type_summary_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.5,83.9,94,97,90
District,77.0,81.0,67,81,54
